In [40]:
import numpy as np

### Randomized sample importance sampling

Because traditional importance sampling seems to reap only marginal results this new approach seeks to identify the dangerous areas within parameter space and increase samples around them.

For now lets assume a 2-dimensional parameter space - With a radius 1 circle in the center as its "danger zone".

In [41]:
def isDangerZone(x):
    d = np.sqrt( x[0] * x[0] +  x[1] * x[1])
    if d <= 1:
        return True
    return False

### The idea
#### Searching 

First we generate samples randomly. This is 1) easier to implement for this proof of concept
2) In higher order search spaces grid search will not work anymore.
We then evaluate all these samples to find those, which are dangerous. Around these samples, more samples are generated.

In [42]:
gridSize = 10

def generateRandom(size):
    return np.random.random() * size - size*0.5

def generateSamples(sampleSize):
    samples = []
    for i in range(0, sampleSize):
        samples.append([generateRandom(gridSize), generateRandom(gridSize)])
    return samples

In [43]:
def getDangerousSamples(samples):
    return list(filter(isDangerZone,samples))
    

#### Sampling

Now that the dangerous parameter areas have been found we can continue importance sampling them:

In [44]:
def randomSamples(size):
    size = 100000
    samples = generateSamples(size)
    dangerousSamples = getDangerousSamples(samples)
    return(len(dangerousSamples)/size)


Finding the importance distribution: I am unsure how to do this though there are some interesting papers about it
Since the pdf of the function is equal in this example it remains a problem for future me ;)

## Sampling around the dangerous samples

When sampling around the dangerous samples, an importance distribution is used. In this sense we are importance sampling for every dangerous sample

In [97]:
def sampleAroundPoint(sampleSize, point):
    samples = []
    for i in range(0, int(sampleSize)):

        samples.append([point[0] + generateRandom(0.01),point[1] + generateRandom(0.01)])
    return samples
    

In [65]:
def importanceSampling(size):
    inital_frac = 0.1
    #importance sampling
    #Intial samples
    init_samples = inital_frac * size
    init_samples = generateSamples(size)
    init_dangerous_samples = getDangerousSamples(init_samples)

    remainingSamples = size*(1-inital_frac)
    dangerousSampleSize = len(init_dangerous_samples)
    importanceSamplePerDangerousSample = remainingSamples/dangerousSampleSize

    dangerousSamples = 0

    for s in init_dangerous_samples:
        dangerousSamples += len(getDangerousSamples(sampleAroundPoint(importanceSamplePerDangerousSample, s)))
    print(dangerousSamples)
    dangerousSamples *= 1/(np.pi)
    return dangerousSamples/remainingSamples


In [107]:
print(importanceSampling(10000))
print(randomSamples(10000))

8803
0.31134243645287885
0.03123
